In [1]:
import pandas as pd
import geopandas
import json
import os
from shapely.geometry import Point
import difflib
from geopy.distance import vincenty
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

%matplotlib inline
inline_rc = dict(mpl.rcParams)

In [2]:
#in_folder = "audits/2018_07/data/"
out_folder = "audits/2019_05/images/"
in_folder = "../data/"
#out_folder = "/tmp"

Fichier des tracés:
    https://opendata.stif.info/explore/dataset/bus_lignes/export/
    
Fichier pour faire la correspondance :
    https://opendata.stif.info/explore/dataset/referentiel-des-lignes-stif/export/

In [3]:
opendata_ref = pd.read_csv("ref_data/referentiel-des-lignes-stif.csv",delimiter=";")
#opendata_ref

In [4]:
gtfs_lines = pd.read_csv(os.path.join(in_folder, "gtfs_routes.txt"), dtype=str)
gtfs_lines = gtfs_lines[gtfs_lines['route_type'] == '3']

In [ ]:
opendata_tracks = pd.read_csv("ref_data/bus_lignes.csv",delimiter=";")

opendata_tracks.dropna(inplace=True)
opendata_tracks.drop_duplicates('ID_GroupOfLines',inplace=True)

opendata_tracks = opendata_tracks.merge(opendata_ref, how='inner', on='ID_GroupOfLines')

opendata_tracks = opendata_tracks[['Geo Shape', 'ShortName_Line', 'TransportMode', 'NetworkName', 'ExternalCode_Line' ]]

opendata_tracks = opendata_tracks[opendata_tracks['TransportMode']=='bus']
#opendata_tracks.head()

In [ ]:
# on prépare les tracés pour analyse
from functools import partial
import pyproj
from shapely import wkt
from shapely.geometry import shape
from shapely.ops import transform

project = partial(
    pyproj.transform,
    pyproj.Proj(init='EPSG:4326'),
    pyproj.Proj(init='EPSG:2154'))

def geojson_to_wkt(row):
    if pd.isna(row['Geo Shape']):
        wkt_ = wkt.loads('POINT (2.482068 49.0327237)')
    else :     
        wkt_ = shape(json.loads(row['Geo Shape']))
    return transform(project, wkt_) # on projette, pour avoir plus tard la longueur en mètres

opendata_tracks['geom'] = opendata_tracks.apply(geojson_to_wkt, axis=1)
opendata_lines_geo = geopandas.GeoDataFrame(opendata_tracks, geometry=opendata_tracks.geom)
#opendata_lines_geo

In [ ]:
#opendata_lines_geo.plot()

In [ ]:
opendata_lines_geo['line_length'] = opendata_lines_geo.length/1000
opendata_lines_geo[['line_length', 'ShortName_Line','NetworkName', 'TransportMode', 'ExternalCode_Line','geom' ]]

print("Nombre de lignes avec tracé : {}".format(len(opendata_lines_geo)))

tt = len(opendata_lines_geo) * 100.0 / len(gtfs_lines)
"Pourcentage de lignes qui ont un tracé : {}".format(tt)

Nombre de lignes avec tracé : 1709


'Pourcentage de lignes qui ont un tracé : 90.51906779661017'

## OSM

In [ ]:
osm_lines = pd.read_csv(os.path.join(in_folder, "osm-transit-extractor_lines.csv"), dtype=str)
osm_lines = osm_lines[osm_lines['mode'] == 'bus']


osm_lines = osm_lines[['line_id', 'name', 'osm:ref:FR:STIF:ExternalCode_Line','code', 'colour', 'operator', 'network','shape']].copy()
#osm_lines.head()

In [ ]:
def osm_shape_to_wkt(row):
    if pd.isna(row['shape']):
        wkt_ = wkt.loads('POINT (2.482068 49.0327237)')
    else :
        wkt_ = wkt.loads(row['shape'])
    return transform(project, wkt_) # on projette, pour avoir plus tard la longueur en mètres

osm_lines['geom'] = osm_lines.apply(osm_shape_to_wkt, axis=1)
osm_lines_geo = geopandas.GeoDataFrame(osm_lines, geometry=osm_lines.geom)


In [ ]:
#osm_lines_geo.plot()

In [ ]:
osm_lines_geo['line_length'] = (osm_lines_geo.length ) /1000
osm_lines_geo_ok = osm_lines_geo[osm_lines_geo['line_length'] >= 0.2].copy() 
osm_lines_geo_ok[['line_id', 'name', 'network','osm:ref:FR:STIF:ExternalCode_Line', 'line_length']]

print("Nombre de lignes avec tracé : {}".format(len(osm_lines_geo_ok)))

tt = len(osm_lines_geo_ok) * 100.0 / len(osm_lines_geo)
"Pourcentage des lignes d'OSM qui ont un tracé : {}".format(tt)

In [ ]:
mpl.rcParams.update(inline_rc) # reset du style 

width = 0.5
ind = np.arange(1)

#fig, ax = plt.subplots(figsize=(6, 8))
fig = plt.figure(figsize=(20, 8))
ax = fig.add_subplot(121)

ax.bar(ind , [len(opendata_lines_geo)],width/4,  color='SkyBlue', label='tracés opendata')
ax.bar(ind + width/4, [len(osm_lines_geo_ok)], width/4, color='IndianRed', label='tracés OSM')
ax.plot([ind - width/8, ind + width/4, ind + width], [len(gtfs_lines),len(gtfs_lines),len(gtfs_lines) ], label="GTFS")
ax.set_xticks(ind)
ax.set_title('Nombre de lignes avec tracés', 
             fontname='Ubuntu', fontsize=16,
             fontstyle='normal', fontweight='normal')
ax.set_ylabel('Nombre de lignes', fontsize=16)
ax.legend()
ax.legend(loc="lower right")

ax = fig.add_subplot(122)
ax.bar(ind , [opendata_lines_geo['line_length'].sum()],width/4,  color='SkyBlue', label='opendata')
ax.bar(ind + width/4, [osm_lines_geo_ok['line_length'].sum()], width/4, color='IndianRed', label='OSM')
ax.plot([ind - width/8, ind + width/4, ind + width], [45000,45000,45000], color="white")

ax.set_xticks(ind)
ax.set_title('Nombre de kilomètres de tracés', 
             fontname='Ubuntu', fontsize=16,
             fontstyle='normal', fontweight='normal')
ax.set_ylabel('Nombre de km', fontsize=16)
ax.legend()
ax.legend(loc="lower right")

plt.savefig(os.path.join(out_folder,'20_traces_de_lignes.png'), bbox_inches='tight')

In [ ]:
tt = opendata_lines_geo['line_length'].sum()
print("Nb de km de lignes dans l'opendata: {}".format(tt))

tt = osm_lines_geo_ok['line_length'].sum()
print("Nb de km de lignes dans OSM: {}".format(tt))

# On fusionne pour faire des comparaisons

In [ ]:
for c in opendata_lines_geo.columns:
    new_col = "opendata_" + c
    opendata_lines_geo.rename(columns={c: new_col}, inplace=True)
for c in osm_lines_geo_ok.columns:
    new_col = "osm_" + c
    osm_lines_geo_ok.rename(columns={c: new_col}, inplace=True)

In [ ]:
lines_geo = opendata_lines_geo.merge(osm_lines_geo_ok, how='inner', right_on='osm_osm:ref:FR:STIF:ExternalCode_Line', left_on='opendata_ExternalCode_Line')
#lines_geo[lines_geo['osm_network']=='Comète'][["opendata_line_length", "osm_line_length", "osm_code"]]
tt = len(lines_geo) * 100.0 / len(gtfs_lines)
print("Pourcentage de lignes comparées : {}".format(tt))


In [ ]:
networks_km = pd.DataFrame(lines_geo.groupby(['opendata_NetworkName'])['osm_line_length', 'opendata_line_length'].sum()).reset_index()

networks_km['diff'] = networks_km['opendata_line_length'] - networks_km['osm_line_length']
#networks_km[['diff']].hist()

In [ ]:
limit_km = 10
opendata_longuer_than_osm = networks_km[networks_km['diff']>limit_km]
osm_longuer_than_opendata = networks_km[networks_km['diff']<-limit_km]
opendata_equals_osm = networks_km[(networks_km["diff"] <= limit_km) & (networks_km["diff"] >= -limit_km)]

portion_osm_longuer = len(osm_longuer_than_opendata)/len(networks_km) * 100
portion_od_longuer = len(opendata_longuer_than_osm)/len(networks_km) * 100
portion_equals = len(opendata_equals_osm)/len(networks_km) * 100
sizes = [portion_osm_longuer, portion_od_longuer, portion_equals]
labels = 'Plus de km dans OSM', 'Plus de km dans l\'open data officiel', 'Même nombre de km'

explode = (0.2, 0.1, 0)  

fig1, ax1 = plt.subplots()
ax1.set_title('Nombre de km de tracé par réseau dans les deux sources (à 10 km près)')
ax1.pie(sizes, explode=explode, 
        labels=labels, 
        autopct='%1.1f%%',
        shadow=True, startangle=90)
ax1.axis('equal') 

plt.savefig(os.path.join(out_folder, '21_km_lines_by_network_pie.png'), bbox_inches='tight')

In [ ]:
#osm_longuer_than_opendata.head()

est-ce qu'il y a des lignes dans OSM qu'on n'a pas dans l'open data ?

In [ ]:
lines_geo_byosm = opendata_lines_geo.merge(osm_lines_geo_ok, how='right', right_on='osm_osm:ref:FR:STIF:ExternalCode_Line', left_on='opendata_ExternalCode_Line')
lines_geo_byosm_alone = lines_geo_byosm[lines_geo_byosm['opendata_ShortName_Line'].isnull()]
lines_in_osm_but_not_in_opendata = lines_geo_byosm_alone[lines_geo_byosm_alone['osm_osm:ref:FR:STIF:ExternalCode_Line'].notnull()]
print("{} lignes avec tracés et code STIF de ligne qu'on ne retrouve pas dans le jeu open data".format(len(lines_in_osm_but_not_in_opendata)))
lines_in_osm_but_not_in_opendata.groupby(['osm_network'])['osm_line_length'].count()
#lines_in_osm_but_not_in_opendata[lines_in_osm_but_not_in_opendata['osm_network']=="RATP"]